In [ ]:
# Cell 1: Imports and Path Setup
import sys
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy
from datetime import datetime
from tqdm import tqdm


sys.path.insert(0, "/home/azhuravl/work/TrajectoryCrafter")
# os.chdir(trajcrafter_path)

import inference_orbits

sys.path.insert(0, "/home/azhuravl/work/TrajectoryCrafter/notebooks/28_08_25_trajectories")

import core, trajectory_generation, viser_utils

In [ ]:
# reload core, trajectory_generation, viser_utils with importlib

import importlib

importlib.reload(core)
importlib.reload(trajectory_generation)
importlib.reload(viser_utils)

In [ ]:
# Cell 2: Argument Setup
# Create opts manually for notebook use
parser = inference_orbits.get_parser()
opts_base = parser.parse_args([
    '--video_path', './test/videos/0-NNvgaTcVzAG0-r.mp4',  # Change this path
    '--radius', '1.0',
    '--device', 'cuda:0'
])

# Set common parameters
opts_base.weight_dtype = torch.bfloat16
opts_base.camera = "target"
opts_base.mode = "gradual"
opts_base.mask = True
opts_base.target_pose = [0, 90, opts_base.radius, 0, 0]  # right_90 example
opts_base.exp_name = f"test"

In [ ]:
# Cell 4: Run Visualization
# Initialize visualization TrajCrafter
print("Initializing TrajCrafter for visualization...")
vis_crafter = core.TrajCrafterVisualization(opts_base)

# Extract scene data
print("Extracting scene data...")
scene_data = vis_crafter.extract_scene_data(opts_base)

# Create warper for 3D point extraction
print("Creating 3D point cloud from all frames...")
vis_warper = core.VisualizationWarper(device=opts_base.device)

# Extract points from all frames
all_points_3d = []
all_colors_rgb = []

num_frames = scene_data['frames_tensor'].shape[0]
for i in tqdm(range(num_frames), desc="Processing frames"):
    frame_data = {
        'frame': scene_data['frames_tensor'][i:i+1],
        'depth': scene_data['depths'][i:i+1], 
        'pose_source': scene_data['pose_source'][i:i+1],
        'intrinsics': scene_data['intrinsics'][i:i+1],
    }
    
    points_3d_frame, colors_rgb_frame = vis_warper.extract_3d_points_with_colors(
        frame_data['frame'],
        frame_data['depth'], 
        frame_data['pose_source'],
        frame_data['intrinsics'],
        subsample_step=20  # Increased for performance with multiple frames
    )
    
    if points_3d_frame.shape[0] > 0:  # Only add if we have valid points
        all_points_3d.append(points_3d_frame)
        all_colors_rgb.append(colors_rgb_frame)

# Concatenate all points
if all_points_3d:
    points_3d = torch.cat(all_points_3d, dim=0)
    colors_rgb = torch.cat(all_colors_rgb, dim=0)
    print(f"Generated {points_3d.shape[0]} 3D points from {len(all_points_3d)} frames")
else:
    print("No valid 3D points extracted!")
    points_3d = None
    colors_rgb = None

print(f"Camera trajectory: {scene_data['pose_target'].shape[0]} poses")

In [ ]:
# Cell 1: Create Viser Server (run once)
import viser

# Check if server already exists and stop it
try:
    if 'viser_server' in globals() and viser_server is not None:
        print("Stopping existing server...")
        viser_server.stop()
        del viser_server
except:
    pass

# Create new server
print("Creating new Viser server on port 8080...")
viser_server = viser.ViserServer(port=8080)
print("Server started successfully!")

In [ ]:
# Setup scene
viser_utils.setup_viser_scene(viser_server, scene_data)

# Show first frame
viser_utils.animate_frame(viser_server, vis_warper, scene_data, 0)

# Add controls
viser_utils.add_animation_controls(viser_server, vis_warper, scene_data)
viser_utils.add_point_size_control(viser_server, points_3d, colors_rgb)
viser_utils.add_camera_controls(viser_server)
viser_utils.add_trajectory_controls(viser_server, scene_data, vis_crafter, opts_base)

In [ ]:
# list all  scene elements
viser_server.scene.list_elements()

In [ ]:
viser_server.scene.remove_by_name("/trajectory")

In [ ]:
#########################################################################
# Camera trajectory visualization
##########################################################################

In [ ]:
# Extract scene data (same as your existing code)
scene_data = vis_crafter.extract_scene_data(opts_base)


In [ ]:
# Generate different circular motions
# horizontal_circle = generate_circular_scene_data(vis_crafter, opts_base, scene_data, 'horizontal')
vertical_circle = generate_circular_scene_data(vis_crafter, opts_base, scene_data, 'vertical_xz')

# Use in your existing Viser visualization
# update_trajectory_visualization(viser_server, horizontal_circle)
update_trajectory_visualization(viser_server, vertical_circle)

print("Circular trajectories generated!")
print("Available types: horizontal, vertical_xz, vertical_yz, tilted")

In [ ]:
# Generate and print trajectory numbers for a few presets
target_poses = {
    "Right 90°": [0, 90, 1, 0, 0],
    "Full Circle": [0, 360, 1, 0, 0],
    "Pull Back": [0, 0, 3, 0, 0]
}

new_data = generate_new_trajectory(
    vis_crafter, opts_base,
    target_poses["Right 90°"],
    scene_data
    )

positions = new_data['pose_target'].cpu().numpy()[:, :3, 3]
